In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train_extra = pd.read_csv('./data/train_new.csv')
test_extra = pd.read_csv('./data/test_new.csv')

duplicate_train = [c for c in train_extra.columns if c in train.columns]
duplicate_test = [c for c in test_extra.columns if c in test.columns]
train_extra = train_extra.drop(columns=duplicate_train)
test_extra = test_extra.drop(columns=duplicate_test)

train = pd.concat([train.reset_index(drop= True), train_extra], axis = 1)
test = pd.concat([test.reset_index(drop= True), test_extra], axis = 1)

In [2]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor 
from catboost import CatBoostRegressor

In [3]:
print(train.shape, test.shape)
print(list(train.columns))
print(list(test.columns))

(80000, 19) (15996, 18)
['time', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'Y1', 'Y2', 'O', 'P']
['id', 'time', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']


In [4]:
all_columns = list(train.columns)
features = []
for col in train.columns:
    if col not in ["Y1", "Y2", "id", "time"]:
        features.append(col)

In [5]:
print(features)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']


In [6]:
train_targets = train.drop(columns=["Y1", "Y2"])
test_ids = test.drop(columns=["id"])

full = pd.concat([train_targets, test_ids], ignore_index = True)

In [7]:
print(full.shape)

(95996, 17)


In [8]:
full = full.sort_values("time").reset_index(drop = True)

In [9]:
lag_list = [1, 2, 3, 5, 10]

In [10]:
for feature in features:
    series = full[feature]
    for k in lag_list:
        col_name = f"{feature}_lag{k}"
        full[col_name] = series.shift(k)

In [11]:
example = features[0]
expected = f"{example}_lag1"
if expected not in full.columns:
    print("Missing")

In [12]:
for feature in features:
    s = full[feature]
    full[f"{feature}_diff1"] = s.diff(1)
    full[f"{feature}_diff5"] = s.diff(5)

C:\Users\anish\AppData\Local\Temp\ipykernel_5600\2075051173.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full[f"{feature}_diff1"] = s.diff(1)
C:\Users\anish\AppData\Local\Temp\ipykernel_5600\2075051173.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full[f"{feature}_diff5"] = s.diff(5)
C:\Users\anish\AppData\Local\Temp\ipykernel_5600\2075051173.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all co

In [13]:
r_windows = [5, 20]
for feature in features:
    s = full[feature]
    for w in r_windows:
        full[f"{feature}_rmean{w}"] = s.rolling(window = w, min_periods= 1).mean()


C:\Users\anish\AppData\Local\Temp\ipykernel_5600\3787376751.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full[f"{feature}_rmean{w}"] = s.rolling(window = w, min_periods= 1).mean()
C:\Users\anish\AppData\Local\Temp\ipykernel_5600\3787376751.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full[f"{feature}_rmean{w}"] = s.rolling(window = w, min_periods= 1).mean()
C:\Users\anish\AppData\Local\Temp\ipykernel_5600\3787376751.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fr

In [14]:
for feature in features:
    s = full[feature]
    for w in r_windows:
        full[f"{feature}_rstd{w}"] = s.rolling(window=w, min_periods=1).std().fillna(0.0)

for feature in features:
    s = full[feature]
    full[f"{feature}_rmin5"] = s.rolling(window = 5, min_periods = 1).min()
    full[f"{feature}_rmax5"] = s.rolling(window = 5, min_periods = 1).max()

C:\Users\anish\AppData\Local\Temp\ipykernel_5600\3105793696.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full[f"{feature}_rstd{w}"] = s.rolling(window=w, min_periods=1).std().fillna(0.0)
C:\Users\anish\AppData\Local\Temp\ipykernel_5600\3105793696.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full[f"{feature}_rstd{w}"] = s.rolling(window=w, min_periods=1).std().fillna(0.0)
C:\Users\anish\AppData\Local\Temp\ipykernel_5600\3105793696.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result 

In [15]:
full = full.copy()

In [16]:
no_train_rows = len(train)
train_fe = full.iloc[:no_train_rows].copy()
test_fe = full.iloc[no_train_rows:].copy()

In [17]:
print(train_fe.shape)
print(test_fe.shape)

(80000, 225)
(15996, 225)


In [18]:
train_time_max = train_fe["time"].max()
test_time_min = test_fe["time"].min()

In [19]:
print(int(train_time_max))
print(int(test_time_min))

80004
80005


In [20]:
for col in train_fe.columns:
    if col != "time":
        train_fe[col] = np.tanh(train_fe[col])
        test_fe[col] = np.tanh(test_fe[col])

In [21]:
max_lag = 10
train_xy = train_fe.iloc[max_lag:].copy()
train_xy[["Y1","Y2"]] = train[["Y1","Y2"]].iloc[max_lag:].values

In [22]:
x = train_xy.drop(columns=["Y1", "Y2", "time"])
y1 = train_xy["Y1"].values
y2 = train_xy["Y2"].values

In [23]:
print(x.shape)
print(len(y1))
print(len(y2))

(79990, 224)
79990
79990


In [24]:
x = x.astype(np.float32)

In [25]:
tscv = TimeSeriesSplit(n_splits = 5)
fold = 0
for index, value in tscv.split(x):
    train_end = int(train_xy.iloc[index]["time"].max())
    value_start = int(train_xy.iloc[value]["time"].min())
    value_end = int(train_xy.iloc[value]["time"].max())
    fold += 1

In [26]:
print(train_end)
print(value_start)

66673
66674


In [27]:
def cv(x, y, splits = 5):
    tscv2 = TimeSeriesSplit(n_splits= splits)
    r2_scores = []
    hgb_params = dict(max_iter = 300, learning_rate = 0.06, max_leaf_nodes = 128, min_samples_leaf= 10, l2_regularization=0.0, random_state= 40)
    et_params = dict(n_estimators = 1200, min_samples_leaf = 3, max_features = 0.7, bootstrap = True, n_jobs = -1, max_samples = 0.9, random_state = 40, max_depth = None)

    fold_id = 0
    for index, value in tscv2.split(x):
        x_train = x.iloc[index]
        x_value = x.iloc[value]
        y_train = y[index]
        y_value = y[value]

        hgb = HistGradientBoostingRegressor(**hgb_params)
        et = ExtraTreesRegressor(**et_params)

        hgb.fit(x_train, y_train)
        et.fit(x_train, y_train)

        predict_hgb = hgb.predict(x_value)
        predict_et = et.predict(x_value)

        predict_b = 0.5 * predict_hgb + 0.5 * predict_et
        fold_r2 = r2_score(y_value, predict_b)
        r2_scores.append(fold_r2)

        fold_id += 1
    r2_scores = np.array(r2_scores)
    print(r2_scores.mean())
    return r2_scores


In [28]:
r2_y1 = cv(x, y1, splits = 5)
r2_y2 = cv(x, y2, splits = 5)
avg_cv = (r2_y1.mean() + r2_y2.mean()) / 2

0.7361159556081642
0.6598891708050927


In [29]:
print((r2_y1[-1] + r2_y2[-1]) / 2)

0.6355930793311155


In [30]:
x_test = test_fe.drop(columns = ["time"]).copy()
x_test = x_test[x.columns]

x_test = x_test.astype(np.float32)

train_cols = list(x.columns)
test_cols = list(x_test.columns)

def cv_single(x, y, model, splits = 5):
    tscv3 = TimeSeriesSplit(n_splits= splits)
    scores = []
    for index, value in tscv3.split(x):
        m = model.__class__(**model.get_params())
        m.fit(x.iloc[index], y[index])
        scores.append(r2_score(y[value], m.predict(x.iloc[value])))
    return np.array(scores)

hgb_cv = HistGradientBoostingRegressor(max_iter = 300, learning_rate = 0.06, max_leaf_nodes = 128, min_samples_leaf= 10, l2_regularization=0.0, random_state= 40)
et_cv = ExtraTreesRegressor(n_estimators = 1200, min_samples_leaf = 3, max_features = 0.7, bootstrap = True, n_jobs = -1, max_samples = 0.9, random_state = 40, max_depth = None)

r2_hgb_y1 = cv_single(x, y1, hgb_cv, splits = 5)
r2_et_y1 = cv_single(x, y1, et_cv, splits = 5)

r2_hgb_y2 = cv_single(x, y2, hgb_cv, splits = 5)
r2_et_y2 = cv_single(x, y2, et_cv, splits = 5)


In [31]:
lgbm_params = dict(
    n_estimators = 1200, learning_rate = 0.03,
    num_leaves = 63, max_depth = -1,
    subsample = 0.9, colsample_bytree = 0.8,
    reg_lambda = 1.0, reg_alpha = 0.0,
    min_child_samples = 20, n_jobs = -1, random_state = 40
)

xgb_params = dict(
    n_estimators = 1200, learning_rate = 0.03,
    max_depth = 8, subsample = 0.9, colsample_bytree = 0.8,
    reg_lambda = 1.0, reg_alpha = 0.0,
    min_child_weight = 2, tree_method = "hist", n_jobs = -1, random_state = 40
)

cat_params = dict(
    depth = 8, learning_rate = 0.03, iterations = 1200,
    l2_leaf_reg = 3.0, loss_function = "RMSE",
    random_seed = 40, verbose = False
)

In [32]:
cv_scores_y1 = {"hgb": r2_hgb_y1.mean(), "et": r2_et_y1.mean()}
cv_scores_y2 = {"hgb": r2_hgb_y2.mean(), "et": r2_et_y2.mean()}

In [33]:
r2_lgbm_y1 = cv_single(x, y1, LGBMRegressor(**lgbm_params), splits = 5).mean()
r2_lgbm_y2 = cv_single(x, y2, LGBMRegressor(**lgbm_params), splits = 5).mean()
cv_scores_y1["lgbm"] = r2_lgbm_y1
cv_scores_y2["lgbm"] = r2_lgbm_y2

r2_xgb_y1 = cv_single(x, y1, XGBRegressor(**xgb_params), splits = 5).mean()
r2_xgb_y2 = cv_single(x, y2, XGBRegressor(**xgb_params), splits = 5).mean()
cv_scores_y1["xgb"] = r2_xgb_y1
cv_scores_y2["xgb"] = r2_xgb_y2

r2_cat_y1 = cv_single(x, y1, CatBoostRegressor(**cat_params), splits = 5).mean()
r2_cat_y2 = cv_single(x, y2, CatBoostRegressor(**cat_params), splits = 5).mean()
cv_scores_y1["cat"] = r2_cat_y1
cv_scores_y2["cat"] = r2_cat_y2


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011818 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50648
[LightGBM] [Info] Number of data points in the train set: 13335, number of used features: 224
[LightGBM] [Info] Start training from score 0.005911
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014461 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50700
[LightGBM] [Info] Number of data points in the train set: 26666, number of used features: 224
[LightGBM] [Info] Start training from score -0.001769
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023111 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50722
[LightGBM] [Info] Number of data points in the train set: 39997, number of used features: 224
[LightGBM] [Info] St

In [34]:
use_m_y1 = [name for name, s in cv_scores_y1.items() if s > 0.0]
use_m_y2 = [name for name, s in cv_scores_y2.items() if s > 0.0]

In [35]:
print(use_m_y1)
print(use_m_y2)

['hgb', 'et', 'lgbm', 'xgb', 'cat']
['hgb', 'et', 'lgbm', 'xgb', 'cat']


In [36]:
# Y1 final models
final_hgb_y1 = HistGradientBoostingRegressor(max_iter = 700, learning_rate = 0.06, max_leaf_nodes = 128, min_samples_leaf= 10, l2_regularization=0.0, random_state= 40).fit(x, y1)
final_et_y1 = ExtraTreesRegressor(n_estimators = 1200, min_samples_leaf = 3, max_features = 0.7, bootstrap = True, n_jobs = -1, max_samples = 0.9, random_state = 40, max_depth = None).fit(x, y1)

# Y2 final models
final_hgb_y2 = HistGradientBoostingRegressor(max_iter = 700, learning_rate = 0.06, max_leaf_nodes = 128, min_samples_leaf= 10, l2_regularization=0.0, random_state= 40).fit(x, y2)
final_et_y2 = ExtraTreesRegressor(n_estimators = 1200, min_samples_leaf = 3, max_features = 0.7, bootstrap = True, n_jobs = -1, max_samples = 0.9, random_state = 40, max_depth = None).fit(x, y2)

models_y1 = {"hgb": final_hgb_y1, "et": final_et_y1}
models_y2 = {"hgb": final_hgb_y2, "et": final_et_y2}

In [37]:
models_y1["lgbm"] = LGBMRegressor(**lgbm_params).fit(x,y1)
models_y2["lgbm"] = LGBMRegressor(**lgbm_params).fit(x,y2)

models_y1["xgb"] = XGBRegressor(**xgb_params).fit(x,y1)
models_y2["xgb"] = XGBRegressor(**xgb_params).fit(x,y2)

models_y1["cat"] = CatBoostRegressor(**cat_params).fit(x,y1)
models_y2["cat"] = CatBoostRegressor(**cat_params).fit(x,y2)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.081316 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50775
[LightGBM] [Info] Number of data points in the train set: 79990, number of used features: 224
[LightGBM] [Info] Start training from score -0.002809
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050684 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50775
[LightGBM] [Info] Number of data points in the train set: 79990, number of used features: 224
[LightGBM] [Info] Start training from score -0.061143


In [38]:
P1 = [models_y1[name].predict(x_test) for name in use_m_y1]
P2 = [models_y2[name].predict(x_test) for name in use_m_y2]
predicted_y1 = np.mean(P1, axis = 0)
predicted_y2 = np.mean(P2, axis = 0)

In [39]:
print(float(np.mean(predicted_y1)))
print(float(np.std(predicted_y1)))
print(float(np.mean(predicted_y2)))
print(float(np.std(predicted_y2)))

0.007413192861159855
0.780470236344406
-0.15071913279524848
0.5855276034697707


In [40]:
sub = pd.DataFrame({
    'id': test['id'],
    'Y1': predicted_y1,
    'Y2': predicted_y2,
})

sub.to_csv('preds.csv', index=False)
print(sub.shape)

(15996, 3)
